In [12]:
import re
import json
from typing import Dict, Optional, List, Tuple
import pymorphy3

# Инициализируем морфологический анализатор
morph = pymorphy3.MorphAnalyzer()


def normalize_text(text: str) -> str:
    """
    Нормализует текст, приводя все слова к начальной форме.
    
    Args:
        text: Исходный текст
        
    Returns:
        Нормализованный текст с словами в начальной форме
    """
    words = text.split()
    normalized_words = []
    
    for word in words:
        clean_word = re.sub(r'[^\w]', '', word)
        if clean_word:
            parsed = morph.parse(clean_word)[0]
            normalized_words.append(parsed.normal_form)
        else:
            normalized_words.append(word)
    
    return ' '.join(normalized_words)


def load_law_aliases_with_morphology():
    """
    Загружает law_aliases.json и создает индекс с нормализованными формами.
    ОПТИМИЗАЦИЯ: предкомпилирует все регулярные выражения один раз.
    
    Returns:
        Tuple[Dict, List]: (нормализованный_алиас -> [(оригинал, law_id)], все алиасы отсортированные)
    """
    with open('law_aliases.json', 'r', encoding='utf-8') as f:
        law_aliases = json.load(f)
    
    normalized_index = {}
    all_aliases = []
    
    for law_id, aliases in law_aliases.items():
        for alias in aliases:
            alias_lower = alias.lower()
            normalized = normalize_text(alias_lower)
            
            if normalized not in normalized_index:
                normalized_index[normalized] = []
            normalized_index[normalized].append((alias_lower, law_id))
            
            # ОПТИМИЗАЦИЯ: предкомпилируем регулярное выражение один раз
            # Используем границы слов только для буквенно-цифровых символов
            escaped_alias = re.escape(alias_lower)
            
            # Добавляем границу слова в начале, только если алиас начинается с буквы/цифры
            if re.match(r'[\w]', alias_lower):
                pattern = r'\b' + escaped_alias
            else:
                pattern = escaped_alias
            
            # Добавляем границу слова в конце, только если алиас заканчивается буквой/цифрой
            if re.search(r'[\w]$', alias_lower):
                pattern = pattern + r'\b'
            
            compiled_pattern = re.compile(pattern)
            
            all_aliases.append({
                'original': alias_lower,
                'normalized': normalized,
                'law_id': law_id,
                'length': len(alias_lower),
                'word_count': len(alias_lower.split()),
                'compiled_pattern': compiled_pattern  # Сохраняем скомпилированный паттерн
            })
    
    all_aliases.sort(key=lambda x: x['length'], reverse=True)
    
    return normalized_index, all_aliases


def find_law_in_text(text: str, normalized_index: Dict, all_aliases: List[Dict]) -> Optional[str]:
    """
    Ищет упоминание кодекса в тексте с учетом склонений.
    
    Args:
        text: Текст для поиска
        normalized_index: Индекс нормализованных форм
        all_aliases: Список всех алиасов
        
    Returns:
        law_id или None
    """
    text_lower = text.lower()
    
    # Сначала пробуем прямое совпадение (быстрее и точнее)
    # ОПТИМИЗАЦИЯ: используем предкомпилированные регулярные выражения
    for alias_data in all_aliases:
        # Используем уже скомпилированный паттерн - в разы быстрее!
        if alias_data['compiled_pattern'].search(text_lower):
            return alias_data['law_id']
    
    # Если прямого совпадения нет, пробуем с нормализацией
    # Разбиваем текст на фразы (последовательности слов)
    word_sequences = re.finditer(r'[а-яёА-ЯЁ\w\s]+', text_lower)
    
    for seq_match in word_sequences:
        sequence = seq_match.group()
        words = sequence.split()
        
        max_window = min(10, len(words))
        
        for window_size in range(max_window, 0, -1):
            for i in range(len(words) - window_size + 1):
                window = words[i:i + window_size]
                phrase = ' '.join(window)
                
                normalized_phrase = normalize_text(phrase)
                
                if normalized_phrase in normalized_index:
                    # Дополнительная проверка для коротких алиасов
                    matches = normalized_index[normalized_phrase]
                    
                    for original_alias, law_id in matches:
                        # Если алиас короткий (как "НК", "ГК"), требуем точного совпадения
                        if len(original_alias) <= 3:
                            # Проверяем, что фраза в тексте выглядит как аббревиатура
                            if re.search(r'\b' + re.escape(phrase.upper()) + r'\b', text.upper()):
                                return law_id
                        else:
                            return law_id
    
    return None


# Загружаем данные один раз
NORMALIZED_INDEX, ALL_ALIASES = load_law_aliases_with_morphology()


def parse_legal_reference_v2(text: str) -> List[Dict[str, Optional[str]]]:
    """
    Парсит юридический текст и извлекает все упоминания статей, пунктов и подпунктов.
    Версия с поддержкой склонений через pymorphy3 и множественных ссылок.
    
    Args:
        text: Текст с упоминаниями статей, пунктов и/или подпунктов
        
    Returns:
        Список словарей с полями law_id, article, point_article, subpoint_article
    """
    results = []
    
    text_lower = text.lower().strip()
    
    # Поиск law_id с учетом склонений
    law_id = find_law_in_text(text_lower, NORMALIZED_INDEX, ALL_ALIASES)
    
    # Паттерны для поиска статей с перечислениями
    # Ищем конструкции типа: "ст. 929, 681.14 и 1988" или "статья 100, 200 и 300"
    article_list_patterns = [
        r'стать[яиеюй]?\s+((?:\d+(?:[.-]\d+)*(?:\s*,\s*|\s+и\s+|\s+или\s+)?)+)',
        r'ст\.?\s*((?:\d+(?:[.-]\d+)*(?:\s*,\s*|\s+и\s+|\s+или\s+)?)+)',
    ]
    
    # Паттерны для поиска пунктов с перечислениями
    point_list_patterns = [
        r'(?<!под)пункт[аеуыои]?\s+((?:(?:\d+[а-я]?|[а-я])(?:\s*,\s*|\s+и\s+|\s+или\s+)?)+)',
        r'(?<![а-я])п\.?\s*((?:(?:\d+[а-я]?|[а-я])(?:\s*,\s*|\s+и\s+|\s+или\s+)?)+)',
    ]
    
    # Паттерны для поиска подпунктов с перечислениями
    # Подпункты могут быть: буква (я), буква+цифры (я1), или просто цифры (26)
    # Важно: одиночная буква не должна захватываться, если за ней идёт точка (п. = пункт, а не подпункт)
    subpoint_list_patterns = [
        r'подпункт[аеуыои]?\s+((?:(?:[а-я](?!\.)\d*|\d+)(?:\s*,\s*|\s+и\s+|\s+или\s+)?)+)',
        r'подп\.?\s*((?:(?:[а-я](?!\.)\d*|\d+)(?:\s*,\s*|\s+и\s+|\s+или\s+)?)+)',
        r'пп\.?\s*((?:(?:[а-я](?!\.)\d*|\d+)(?:\s*,\s*|\s+и\s+|\s+или\s+)?)+)',
    ]
    
    # Функция для разбора перечисления
    def parse_enumeration(enum_str: str, pattern: str) -> List[str]:
        """Разбивает строку перечисления на отдельные элементы"""
        # Заменяем союзы на запятые для единообразия
        enum_str = re.sub(r'\s+и\s+', ',', enum_str)
        enum_str = re.sub(r'\s+или\s+', ',', enum_str)
        # Разбиваем по запятым
        items = [item.strip() for item in enum_str.split(',')]
        # Фильтруем пустые и оставляем только те, что соответствуют паттерну
        return [item for item in items if item and re.match(pattern, item)]
    
    # Поиск всех статей
    articles_found = []
    for pattern in article_list_patterns:
        matches = re.finditer(pattern, text_lower)
        for match in matches:
            enum_str = match.group(1)
            articles = parse_enumeration(enum_str, r'^\d+(?:[.-]\d+)*$')
            articles_found.extend(articles)
    
    # Поиск всех пунктов
    points_found = []
    for pattern in point_list_patterns:
        matches = re.finditer(pattern, text_lower)
        for match in matches:
            enum_str = match.group(1)
            points = parse_enumeration(enum_str, r'^(?:\d+[а-я]?|[а-я])$')
            # Фильтруем "п" - это сокращение для слова "пункт", а не номер пункта
            points = [p for p in points if p != 'п']
            points_found.extend(points)
    
    # Поиск всех подпунктов
    subpoints_found = []
    for pattern in subpoint_list_patterns:
        matches = re.finditer(pattern, text_lower)
        for match in matches:
            enum_str = match.group(1)
            # Подпункты: буква (я), буква+цифры (я1), или просто цифры (26)
            subpoints = parse_enumeration(enum_str, r'^(?:[а-я]\d*|\d+)$')
            # Фильтруем "п" и "подп" - это сокращения для слова "пункт/подпункт", а не номера
            subpoints = [s for s in subpoints if s not in ['п', 'подп', 'пп']]
            subpoints_found.extend(subpoints)
    
    # Конвертируем law_id из строки в число
    law_id_int = int(law_id) if law_id is not None else None
    
    # Формируем результаты на основе найденных элементов
    # Логика: создаем запись для самого детального уровня перечисления
    # Если есть несколько подпунктов -> для каждого подпункта
    # Иначе, если есть несколько пунктов -> для каждого пункта
    # Иначе, если есть несколько статей -> для каждой статьи
    
    if subpoints_found and len(subpoints_found) > 1:
        # Есть перечисление подпунктов - создаем запись для каждого
        for subpoint in subpoints_found:
            result = {
                "law_id": law_id_int,
                "article": articles_found[0] if articles_found else None,
                "point_article": points_found[0] if points_found else None,
                "subpoint_article": subpoint
            }
            results.append(result)
    elif points_found and len(points_found) > 1:
        # Есть перечисление пунктов - создаем запись для каждого
        for point in points_found:
            result = {
                "law_id": law_id_int,
                "article": articles_found[0] if articles_found else None,
                "point_article": point,
                "subpoint_article": subpoints_found[0] if subpoints_found else None
            }
            results.append(result)
    elif articles_found and len(articles_found) > 1:
        # Есть перечисление статей - создаем запись для каждой
        for article in articles_found:
            result = {
                "law_id": law_id_int,
                "article": article,
                "point_article": points_found[0] if points_found else None,
                "subpoint_article": subpoints_found[0] if subpoints_found else None
            }
            results.append(result)
    elif articles_found or points_found or subpoints_found:
        # Единичное упоминание
        result = {
            "law_id": law_id_int,
            "article": articles_found[0] if articles_found else None,
            "point_article": points_found[0] if points_found else None,
            "subpoint_article": subpoints_found[0] if subpoints_found else None
        }
        results.append(result)
    else:
        # Если ничего не найдено, возвращаем пустую запись
        results.append({
            "law_id": law_id_int,
            "article": None,
            "point_article": None,
            "subpoint_article": None
        })
    
    return results




In [13]:
# Загрузка и выполнение тестов из demo_test_cases.json

def run_tests():
    """
    Загружает тесты из demo_test_cases.json и прогоняет их через parse_legal_reference_v2
    """
    with open('demo_test_cases.json', 'r', encoding='utf-8') as f:
        test_cases = json.load(f)
    
    print("=" * 80)
    print("ТЕСТИРОВАНИЕ")
    print("=" * 80)
    
    total_tests = 0
    passed_tests = 0
    failed_tests = 0
    
    for idx, test_data in enumerate(test_cases, 1):
        text = test_data['text']
        expected_results = test_data['test_cases']
        
        # Получаем результаты от функции
        actual_results = parse_legal_reference_v2(text)
        
        print(f"\nТЕСТ {idx}:")
        print(f"Текст: '{text[:100]}...'")
        print()
        
        # Сравниваем результаты
        is_passed = compare_results(expected_results, actual_results)
        
        total_tests += 1
        if is_passed:
            passed_tests += 1
            print("✅ ТЕСТ ПРОЙДЕН")
        else:
            failed_tests += 1
            print("❌ ТЕСТ НЕ ПРОЙДЕН")
        
        print(f"Найдено ссылок: {len(actual_results)}")
        print("Ожидаемые результаты:")
        print(json.dumps(expected_results, indent=2, ensure_ascii=False))
        print("Полученные результаты:")
        print(json.dumps(actual_results, indent=2, ensure_ascii=False))
        print("-" * 80)
    
    print("\n" + "=" * 80)
    print(f"ИТОГО: {passed_tests}/{total_tests} тестов пройдено")
    print(f"Успешно: {passed_tests}, Провалено: {failed_tests}")
    print("=" * 80)


def compare_results(expected, actual):
    """
    Сравнивает ожидаемые и фактические результаты
    """
    if len(expected) != len(actual):
        return False
    
    # Создаем копии для сортировки
    expected_sorted = sorted(expected, key=lambda x: (
        x.get('law_id') or 0,
        x.get('article') or '',
        x.get('point_article') or '',
        x.get('subpoint_article') or ''
    ))
    
    actual_sorted = sorted(actual, key=lambda x: (
        x.get('law_id') or 0,
        x.get('article') or '',
        x.get('point_article') or '',
        x.get('subpoint_article') or ''
    ))
    
    for exp, act in zip(expected_sorted, actual_sorted):
        if (exp.get('law_id') != act.get('law_id') or
            exp.get('article') != act.get('article') or
            exp.get('point_article') != act.get('point_article') or
            exp.get('subpoint_article') != act.get('subpoint_article')):
            return False
    
    return True


def show_test_details(test_num):
    """
    Показывает детали конкретного теста
    
    Args:
        test_num: Номер теста (1-based)
    """
    with open('demo_test_cases.json', 'r', encoding='utf-8') as f:
        test_cases = json.load(f)
    
    if test_num < 1 or test_num > len(test_cases):
        print(f"Ошибка: тест {test_num} не существует. Доступны тесты 1-{len(test_cases)}")
        return
    
    test_data = test_cases[test_num - 1]
    text = test_data['text']
    expected_results = test_data['test_cases']
    
    print(f"ТЕСТ {test_num}")
    print("=" * 80)
    print(f"Текст:\n{text}\n")
    print("-" * 80)
    
    actual_results = parse_legal_reference_v2(text)
    
    print(f"\nОжидается найти: {len(expected_results)} ссылок")
    print(f"Найдено: {len(actual_results)} ссылок")
    print()
    
    print("ОЖИДАЕМЫЕ РЕЗУЛЬТАТЫ:")
    for i, exp in enumerate(expected_results, 1):
        print(f"{i}. law_id={exp.get('law_id')}, article={exp.get('article')}, "
              f"point={exp.get('point_article')}, subpoint={exp.get('subpoint_article')}")
    
    print("\nПОЛУЧЕННЫЕ РЕЗУЛЬТАТЫ:")
    for i, act in enumerate(actual_results, 1):
        print(f"{i}. law_id={act.get('law_id')}, article={act.get('article')}, "
              f"point={act.get('point_article')}, subpoint={act.get('subpoint_article')}")
    
    print("\n" + "=" * 80)
    
    # Сравнение
    is_passed = compare_results(expected_results, actual_results)
    if is_passed:
        print("✅ ТЕСТ ПРОЙДЕН")
    else:
        print("❌ ТЕСТ НЕ ПРОЙДЕН")
        print("\nРАЗЛИЧИЯ:")
        show_differences(expected_results, actual_results)


def show_differences(expected, actual):
    """
    Показывает различия между ожидаемыми и фактическими результатами
    """
    if len(expected) != len(actual):
        print(f"- Количество ссылок: ожидалось {len(expected)}, получено {len(actual)}")
    
    # Находим отличия
    for i, (exp, act) in enumerate(zip(expected, actual), 1):
        differences = []
        if exp.get('law_id') != act.get('law_id'):
            differences.append(f"law_id: {exp.get('law_id')} -> {act.get('law_id')}")
        if exp.get('article') != act.get('article'):
            differences.append(f"article: {exp.get('article')} -> {act.get('article')}")
        if exp.get('point_article') != act.get('point_article'):
            differences.append(f"point: {exp.get('point_article')} -> {act.get('point_article')}")
        if exp.get('subpoint_article') != act.get('subpoint_article'):
            differences.append(f"subpoint: {exp.get('subpoint_article')} -> {act.get('subpoint_article')}")
        
        if differences:
            print(f"Ссылка {i}: {', '.join(differences)}")


# Запуск всех тестов
run_tests()


ТЕСТИРОВАНИЕ

ТЕСТ 1:
Текст: 'Во время проверки воздушного судна была обнаружена необходимость уточнения некоторых процедур. В соо...'

✅ ТЕСТ ПРОЙДЕН
Найдено ссылок: 1
Ожидаемые результаты:
[
  {
    "law_id": 3,
    "article": "489",
    "point_article": "24",
    "subpoint_article": "и"
  }
]
Полученные результаты:
[
  {
    "law_id": 3,
    "article": "489",
    "point_article": "24",
    "subpoint_article": "и"
  }
]
--------------------------------------------------------------------------------

ТЕСТ 2:
Текст: 'В ходе заседания представитель компании-перевозчика указал на соблюдение всех требований безопасност...'

✅ ТЕСТ ПРОЙДЕН
Найдено ссылок: 5
Ожидаемые результаты:
[
  {
    "law_id": 3,
    "article": "1048.15-8",
    "point_article": "ъ",
    "subpoint_article": "я"
  },
  {
    "law_id": 3,
    "article": "1048.15-8",
    "point_article": "ъ",
    "subpoint_article": "26"
  },
  {
    "law_id": 3,
    "article": "1048.15-8",
    "point_article": "ъ",
    "subpoint_article

In [20]:

show_test_details(5)


ТЕСТ 5
Текст:
Компания, занимающаяся транспортировкой грузов, была подвергнута аудиту налоговыми органами. Было установлено, что она не соблюдала требования, указанные в пп. Х, д, 21, и в п. 26 ст. 1930.7 Бюджетного кодекса РФ. Руководство обязали исправить нарушения в течение месяца, чтобы избежать штрафных санкций.

--------------------------------------------------------------------------------

Ожидается найти: 4 ссылок
Найдено: 4 ссылок

ОЖИДАЕМЫЕ РЕЗУЛЬТАТЫ:
1. law_id=1, article=1930.7, point=26, subpoint=Х
2. law_id=1, article=1930.7, point=26, subpoint=д
3. law_id=1, article=1930.7, point=26, subpoint=21
4. law_id=1, article=1930.7, point=26, subpoint=в

ПОЛУЧЕННЫЕ РЕЗУЛЬТАТЫ:
1. law_id=1, article=1930.7, point=26, subpoint=х
2. law_id=1, article=1930.7, point=26, subpoint=д
3. law_id=1, article=1930.7, point=26, subpoint=21
4. law_id=1, article=1930.7, point=26, subpoint=и

❌ ТЕСТ НЕ ПРОЙДЕН

РАЗЛИЧИЯ:
Ссылка 1: subpoint: Х -> х
Ссылка 4: subpoint: в -> и


In [4]:
# Простой тест для отладки
import re

test_text = "подпп. я, 26, 29, 22, 3 п. ъ ст. 1048.15-8"
text_lower = test_text.lower()

# Проверяем паттерны для пунктов
point_patterns = [
    r'(?<!под)пункт[аеуыои]?\s+((?:(?:\d+[а-я]?|[а-я])(?:\s*,\s*|\s+и\s+|\s+или\s+)?)+)',
    r'(?<![а-я])п\.?\s*((?:(?:\d+[а-я]?|[а-я])(?:\s*,\s*|\s+и\s+|\s+или\s+)?)+)',
]

print("Тестовый текст:", test_text)
print("\nПоиск пунктов:")
for i, pattern in enumerate(point_patterns, 1):
    print(f"\nПаттерн {i}: {pattern}")
    matches = re.finditer(pattern, text_lower)
    for match in matches:
        print(f"  Найдено: '{match.group(0)}' -> группа 1: '{match.group(1)}'")
        print(f"  Позиция: {match.start()}-{match.end()}")

# Проверяем паттерны для подпунктов
subpoint_patterns = [
    r'подпункт[аеуыои]?\s+((?:(?:[а-я](?!\.)\d*|\d+)(?:\s*,\s*|\s+и\s+|\s+или\s+)?)+)',
    r'подп\.?\s*((?:(?:[а-я](?!\.)\d*|\d+)(?:\s*,\s*|\s+и\s+|\s+или\s+)?)+)',
    r'пп\.?\s*((?:(?:[а-я](?!\.)\d*|\d+)(?:\s*,\s*|\s+и\s+|\s+или\s+)?)+)',
]

print("\n\nПоиск подпунктов:")
for i, pattern in enumerate(subpoint_patterns, 1):
    print(f"\nПаттерн {i}: {pattern}")
    matches = re.finditer(pattern, text_lower)
    for match in matches:
        print(f"  Найдено: '{match.group(0)}' -> группа 1: '{match.group(1)}'")
        print(f"  Позиция: {match.start()}-{match.end()}")


Тестовый текст: подпп. я, 26, 29, 22, 3 п. ъ ст. 1048.15-8

Поиск пунктов:

Паттерн 1: (?<!под)пункт[аеуыои]?\s+((?:(?:\d+[а-я]?|[а-я])(?:\s*,\s*|\s+и\s+|\s+или\s+)?)+)

Паттерн 2: (?<![а-я])п\.?\s*((?:(?:\d+[а-я]?|[а-я])(?:\s*,\s*|\s+и\s+|\s+или\s+)?)+)
  Найдено: 'подпп' -> группа 1: 'одпп'
  Позиция: 0-5
  Найдено: 'п. ъ' -> группа 1: 'ъ'
  Позиция: 24-28


Поиск подпунктов:

Паттерн 1: подпункт[аеуыои]?\s+((?:(?:[а-я](?!\.)\d*|\d+)(?:\s*,\s*|\s+и\s+|\s+или\s+)?)+)

Паттерн 2: подп\.?\s*((?:(?:[а-я](?!\.)\d*|\d+)(?:\s*,\s*|\s+и\s+|\s+или\s+)?)+)

Паттерн 3: пп\.?\s*((?:(?:[а-я](?!\.)\d*|\d+)(?:\s*,\s*|\s+и\s+|\s+или\s+)?)+)
  Найдено: 'пп. я, 26, 29, 22, 3' -> группа 1: 'я, 26, 29, 22, 3'
  Позиция: 3-23
